This graph shows (...) 



In [7]:
import pandas as pd
import folium # type: ignore
import os

# Get the current directory and set up the CSV path
current_dir = os.getcwd()
csv_path = os.path.join(current_dir, 'GamingRevenue2023.csv')

# Read the CSV file
df = pd.read_csv(csv_path)

# Format revenue for display and calculate revenue in billions
df['Revenue_Billions'] = (df['Revenue'] / 1e9).round(2)
df['Revenue_Formatted'] = df['Revenue'].apply(lambda x: f"${x:,.0f}")

# Calculate the center of the map based on the states with data
state_coords = {
    'Nevada': [39.5, -116.5],
    'Pennsylvania': [40.5, -77.5],
    'New Jersey': [40.0, -74.5],
    'New York': [43.0, -75.0],
    'Michigan': [44.0, -85.0],
    'Ohio': [40.5, -82.5],
    'Indiana': [40.0, -86.0],
    'Louisiana': [31.0, -92.0],
    'Illinois': [40.0, -89.0],
    'Maryland': [39.0, -76.5],
    'Mississippi': [32.5, -89.5],
    'Iowa': [42.0, -93.5],
    'Missouri': [38.5, -92.5],
    'Massachusetts': [42.0, -71.5],
    'Colorado': [39.0, -105.5],
    'Virginia': [37.5, -78.5],
    'West Virginia': [38.5, -80.5],
    'Rhode Island': [41.5, -71.5],
    'Florida': [28.0, -82.0],
    'Arkansas': [35.0, -92.5],
    'Kansas': [38.5, -98.5],
    'Connecticut': [41.5, -72.5],
    'Arizona': [34.0, -111.5],
    'Delaware': [39.0, -75.5],
    'Tennessee': [36.0, -86.0],
    'New Mexico': [34.5, -106.0],
    'Maine': [45.0, -69.0],
    'Oklahoma': [35.5, -97.5],
    'South Dakota': [44.5, -100.0],
    'Kentucky': [37.5, -85.0],
    'Nebraska': [41.5, -99.5],
    'New Hampshire': [44.0, -71.5],
    'Oregon': [44.0, -120.5],
    'District of Columbia': [38.9, -77.0],
    'Wyoming': [43.0, -107.5],
    'Montana': [47.0, -110.0]
}

# Calculate the center of the map
center_lat = sum(coord[0] for coord in state_coords.values()) / len(state_coords)
center_lon = sum(coord[1] for coord in state_coords.values()) / len(state_coords)

# Create a base map centered on the states with data
m = folium.Map(location=[center_lat, center_lon], zoom_start=4, tiles='CartoDB positron')

# Create a dictionary of state revenues for easy lookup
state_revenues = dict(zip(df['State'], df['Revenue_Formatted']))
state_revenues_billions = dict(zip(df['State'], df['Revenue_Billions']))

# Add US states GeoJSON with proper styling
us_states = os.path.join(current_dir, 'us-states.json')

def get_color(revenue):
    if revenue > 10:
        return '#8B0000'  # Dark red for highest revenue
    elif revenue > 5:
        return '#FFD700'  # Yellow
    elif revenue > 2:
        return '#0000FF'  # Blue
    else:
        return '#006400'  # Dark green

def style_function(feature):
    state_name = feature['properties']['name']
    if state_name in state_revenues_billions:
        revenue = state_revenues_billions[state_name]
        return {
            'fillColor': get_color(revenue),
            'color': 'white',
            'weight': 1,
            'fillOpacity': 0.7,
        }
    return {
        'fillColor': '#ffffff',
        'color': '#ffffff',
        'weight': 0,
        'fillOpacity': 0,
    }

def highlight_function(feature):
    state_name = feature['properties']['name']
    if state_name in state_revenues_billions:
        return {
            'fillColor': '#4169E1',  # Royal blue on hover
            'color': 'white',
            'weight': 2,
            'fillOpacity': 0.8,
        }
    return {
        'fillColor': '#ffffff',
        'color': '#ffffff',
        'weight': 0,
        'fillOpacity': 0,
    }

# Create a dictionary for tooltip content
tooltip_data = {}
for state in df['State']:
    tooltip_data[state] = state_revenues[state]

# Add the GeoJSON layer with hover effects and tooltips
folium.GeoJson(
    us_states,
    name='US States',
    style_function=style_function,
    highlight_function=highlight_function,
    tooltip=None  # Remove the state name tooltip
).add_to(m)

# Add state labels and revenue tooltips
for state in df['State']:
    if state in state_coords:
        # Create a marker with the state name
        marker = folium.Marker(
            location=state_coords[state],
            icon=folium.DivIcon(
                html=f'<div style="font-size: 12px; font-weight: bold; color: black; text-shadow: -1px -1px 0 white, 1px -1px 0 white, -1px 1px 0 white, 1px 1px 0 white;">{state}</div>'
            )
        )
        
        # Add a tooltip with the revenue
        tooltip = folium.Tooltip(
            state_revenues[state],
            style=('background-color: white; color: black; font-family: Arial; font-size: 14px; padding: 8px; border: 1px solid #ccc; border-radius: 4px; box-shadow: 2px 2px 4px rgba(0,0,0,0.1);')
        )
        marker.add_child(tooltip)
        marker.add_to(m)

# Add a title
title_html = '''
<div style="position: fixed; 
            top: 10px; left: 50%;
            transform: translateX(-50%);
            z-index: 9999;
            background-color: white;
            padding: 10px 20px;
            border: 2px solid #3498db;
            border-radius: 5px;
            font-family: Arial;
            font-size: 18px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
    <b>Gaming Revenue by State (2023)</b>
</div>
'''
m.get_root().html.add_child(folium.Element(title_html))

# Add a legend
legend_html = '''
<div style="position: fixed; 
            bottom: 20px; right: 20px;
            z-index: 9999;
            background-color: white;
            padding: 15px;
            border: 2px solid #3498db;
            border-radius: 5px;
            font-family: Arial;
            font-size: 12px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
    <div style="margin-bottom: 10px;"><b>Revenue (in billions)</b></div>
    <div style="display: flex; align-items: center; margin-bottom: 5px;">
        <div style="width: 20px; height: 20px; background-color: #8B0000; margin-right: 10px;"></div>
        <span>> $10B</span>
    </div>
    <div style="display: flex; align-items: center; margin-bottom: 5px;">
        <div style="width: 20px; height: 20px; background-color: #FFD700; margin-right: 10px;"></div>
        <span>$5B - $10B</span>
    </div>
    <div style="display: flex; align-items: center; margin-bottom: 5px;">
        <div style="width: 20px; height: 20px; background-color: #0000FF; margin-right: 10px;"></div>
        <span>$2B - $5B</span>
    </div>
    <div style="display: flex; align-items: center;">
        <div style="width: 20px; height: 20px; background-color: #006400; margin-right: 10px;"></div>
        <span>< $2B</span>
    </div>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Save the map
output_path = os.path.join(current_dir, "gaming_revenue_map.html")
m.save(output_path)

# Open the file in the default web browser
import webbrowser
webbrowser.open('file://' + os.path.abspath(output_path))

print(f"Map has been saved to: {output_path}")
print("The visualization should open in your default web browser.") 

Map has been saved to: /Users/mursal/Documents/Social data analysis and visualization/USgamingstates.github.io/docs/gaming_revenue_map.html
The visualization should open in your default web browser.
